<a href="https://colab.research.google.com/github/5687d/Machine-learning/blob/main/project1/knn-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [127]:

# -*- coding: utf-8 -*-

"""

@author: yh960
reference: https://www.programmersought.com/article/12664496909/
           https://www.kaggle.com/shrutimechlearn/step-by-step-diabetes-classification-knn-detailed
           https://www.w3xue.com/exp/article/20201/73169.html 
"""
import numpy
import csv
import time
import torch

device = torch.device("cuda:0")

# load the train data
def loadTrainData():
     l=[]
     with open('/content/drive/My Drive/lab/train.csv') as file:
          lines = csv.reader(file)
          for line in lines:
              l.append(line)
          l.remove(l[0])
          l=array(l)
          print(l.shape)
          data, label = l[:,1:], l[:,0]
          label = label[:,newaxis]
          data = toInt(data)
          label = toInt(label)
          return data, label

# integer
def toInt(array):
    tensor = torch.from_numpy(array.astype(numpy.int_)).to(device)
    print('toInt: size = ' + str(tensor.size()))
    return tensor
  
# Normalization
#def normalizing(array):
#    m,n = shape(array)
#    for i in range(m):
#        for j in range(n):
#            if array[i,j] != 0:
#                array[i,j]=1
#    return array
  
# load the test data
def loadTestData():
      l=[]         

      with open('/content/drive/My Drive/lab/test.csv') as file:
          lines = csv.reader(file)
          for line in lines:
              l.append(line)
          l.remove(l[0])
          l=array(l)
          testdata=toInt(l)
          return testdata
  
# save the result
def saveResult(result):
      with open('/content/drive/My Drive/lab/knn.csv', 'w', newline='') as myFile:
          myWriter = csv.writer(myFile)
          myWriter.writerow(['ImageId','Label'])
          for i, label in enumerate(result):
              tmp = [i+1, int(label)]
              myWriter.writerow(tmp)



(42000, 785)
toInt: size = torch.Size([42000, 784])
toInt: size = torch.Size([42000, 1])


In [76]:
trainData,trainLabel=loadTrainData()
testData =loadTestData()


toInt: size = torch.Size([42000, 784])
toInt: size = torch.Size([42000, 1])
toInt: size = torch.Size([28000, 784])


In [133]:
  
# kNN classifier
def classify(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = inX.repeat(dataSetSize,1)-dataSet

    # calculate the L2 distance
    spDiffMat = torch.square(diffMat)
    spDistances = torch.sum(spDiffMat,dim=1) 
    distances = spDistances ** 0.5
    sortedDistIndicies = distances.argsort()
    classCount = {}

    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i],0]
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    sortedClassCount = sorted(classCount.items(), key=lambda item:item[1], reverse=True)
    return sortedClassCount[0][0]
  
# main
start = time.time()

errorCount=0
resultList=[]
splitTrainDataSize = 33600
splitTestDataSize = 8400
k=1

splitTrainData = torch.split(trainData, [splitTrainDataSize, splitTestDataSize])
splitTrainLabel = torch.split(trainLabel, [splitTrainDataSize, splitTestDataSize])
for i in range(splitTestDataSize):
    classifierResult = classify(splitTrainData[1][i], splitTrainData[0], splitTrainLabel[0], k)
    #resultList.append(classifierResult)
    if i % 1000 == 0:
        print(time.time()-start)
    #print("the classifier for %d came back with: %d, the actual answer is %d" % (i, classifierResult, splitTrainLabel[1][27]))
    if (classifierResult != splitTrainLabel[1][i]): 
        errorCount += 1



end = time.time()
print (end - start)
print("\nthe total number of errors is: %d" % errorCount)
print("\nthe total error rate is: %f" % (errorCount/float(splitTestDataSize)))


0.008462667465209961
7.584174394607544
15.141709566116333
22.700247764587402
30.277064561843872
37.84251928329468
45.39778780937195
52.95931601524353
60.52288484573364
63.54207134246826

the total number of errors is: 274

the total error rate is: 0.032619


In [134]:
start = time.time()
testDataSize = testData.shape
print(testDataSize)
for i in range(testDataSize[0]):
    classifierResult = classify(testData[i], splitTrainData[0], splitTrainLabel[0], k)
    resultList.append(classifierResult)
    if i % 1000 == 0:
        print(time.time()-start)
    #print("the classifier for %d came back with: %d, the actual answer is %d" % (i, classifierResult, splitTrainLabel[1][i]))
    #if (classifierResult != splitTrainLabel[1][i]): 
        #errorCount += 1


saveResult(resultList)

torch.Size([28000, 784])
0.008121967315673828
7.533229827880859
15.053433895111084
22.574098587036133
30.095207929611206
37.610623598098755
45.12804341316223
52.64662790298462
60.16444683074951
67.68178081512451
75.19727993011475
82.70944619178772
90.22563576698303
97.74164915084839
105.25846242904663
112.77802753448486
120.29771590232849
127.8216860294342
135.34389543533325
142.86970019340515
150.39158749580383
157.90765953063965
165.42207956314087
172.94549202919006
180.4655933380127
187.98699378967285
195.50607705116272
203.02162098884583
